<a href="https://colab.research.google.com/github/gnoparus/bualabs/blob/master/nbs/32b_langchain_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now we will start learning about how to use the LangChain API.

# 1. Import Library

Install OpenAI Library by Uncomment Cell below and Run (Shift + Enter)

In [ ]:
# ! pip install openai

# 2. Authentication

Before we can access and utilize ChatGPT and GPT-4 from OpenAI, we need to register, sign up, and provide the necessary payment information.

## Sign up

Click on the green Sign up button in the link below to register for OpenAI.

https://platform.openai.com/overview

Please enter your credit card information on the Billing page.

https://platform.openai.com/account/billing/payment-methods






## API Key

To generate an API Key, please follow the steps below:

1. Access the link provided below.
2. Click on the "Create new secret key" button.
3. Give a name to the key.
4. Copy the secret key generated and keep it ready to be entered in the "OPENAI_API_KEY" field below.

Link: https://platform.openai.com/account/api-keys

Please make sure to securely store and protect your API Key as it grants access to the OpenAI API.


## Sample API Key 

OPENAI_API_KEY: sk-abcDeFGHijKlMNopQrstuvWxyZ1234567890abcDefhiJklm

Please execute the cell below and enter the secret key.


In [ ]:
import os
from getpass import getpass

token = getpass('Enter OPENAI_API_KEY here: ')
# print ('token is', token)

os.environ["OPENAI_API_KEY"] = token

Enter OPENAI_API_KEY here: ··········


After completing the testing, it is recommended to go back and delete the Secret Key on the page https://platform.openai.com/account/api-keys every time for security purposes. We can generate new keys continuously and without limitations.

#3. Invoke the API 

## 3.1 Invoke the OpenAI API directly.